<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/test_wrds_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

In [1]:
!pip install wrds
import pandas as pd, numpy as np, wrds

# Read datasets

In [2]:
db = wrds.Connection()

Enter your WRDS username [root]:hohn
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
om_name = (db.get_table('optionm','secnmd')
                .groupby(['secid','ticker','issuer'])['effect_date']
                .agg(['min','max'])
                .reset_index().rename({'min':'start','max':'end'},axis=1))

In [4]:
om_name['ticker'] = np.where(om_name['ticker']=='ZZZZ',np.NaN,om_name['ticker'])

Unique ticker list from OptionMetrics

In [5]:
ticker = set(om_name.ticker.dropna().unique())

Update HERE to drop tickers

In [6]:
# ticker.difference_update(set(['I','WSB','SEC']))

In [105]:
sub_2017 = pd.read_parquet('/content/sub2017.parquet.gzip')

In [106]:
sub_2017['title'] = np.where(sub_2017['title'].isin(['[deleted]','[removed]',None]),
                             '', sub_2017['title'])
sub_2017['selftext'] = np.where(sub_2017['selftext'].isin(['[deleted]','[removed]',None]),
                             '', sub_2017['selftext'])

In [107]:
sub_2017['title'] =sub_2017['title'].str.replace('[,.\?:\(\)]',' ',regex=True)
sub_2017['selftext'] =sub_2017['selftext'].str.replace('[,.\?:\(\)]',' ',regex=True)
sub_2017['title'] =sub_2017['title'].str.replace("'s","",regex=True)
sub_2017['selftext'] =sub_2017['selftext'].str.replace("'s","",regex=True)

In [115]:
sub_2017['title'] =  sub_2017['title'].str.split(' ')
sub_2017['selftext'] =  sub_2017['selftext'].str.split(' ')

In [116]:
test_A = sub_2017.head(100)
test_B = sub_2017.tail(100)

In [119]:
def find_ticker(textlist):
    global ticker
    a_set = set('$' + tic for tic in ticker)
    if len(textlist) != 0:
        hit_1 = a_set.intersection(textlist)
        if len(hit_1) > 0:
            return [s[1:] for s in hit_1]
        else:
            num_upper = len([word for word in textlist if word.isupper()])
            if num_upper / len(textlist) > .8:
                return []
            else:
                hit_2 = ticker.intersection(textlist)
                if len(hit_2) > 0:
                    return [s for s in hit_2]
                else:
                    return []

In [122]:
test_A['title_ticker'] = test_A['title'].apply(find_ticker)
test_A['text_ticker'] = test_A['selftext'].apply(find_ticker)
test_B['title_ticker'] = test_B['title'].apply(find_ticker)
test_B['text_ticker'] = test_B['selftext'].apply(find_ticker)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

Additional corrections

* Intersect title and body
* If intersect with $, stop
* Use comments

Variables to capture from posts

* Number of posts
* Combined rewards
* Comments
* Score